### Shallowest path across the river

Your job is to write a function shallowest_path(river) that takes a list of lists of positive ints (or array of arrays, depending on language) showing the depths of the river as shown in the example above and returns a shallowest path (i.e., the maximum depth is minimal) as a list of coordinate pairs (represented as tuples, Pairs, or arrays, depending on language) as described above. If there are several paths that are equally shallow, the function shall return a shortest such path. All depths are given as positive integers.

In [70]:
import heapq
import math
from collections import namedtuple

STEPS = [ [0,1], [1,0], [1,1], [-1,1], [1,-1], [0,-1], [-1,0], [-1,-1] ]

Point = namedtuple('Point', ['id', 'y', 'x', 'depth', 'steps'])

class PathQueue(object):
    def __init__(self):
        self._points = []

    def push(self, point):
        heapq.heappush(self._points, (point.depth, point.steps, point.id, point))

    def pop(self):
        return heapq.heappop(self._points)[-1]
    
    def is_not_empty(self):
        return len(self._points) > 0;
    
class PathFinder(object):
    
    def __init__(self, river):
        self.river = river
        self.y_size = len(river)
        self.x_size = len(river[0])
        self.maze = [  [ self.create_init_point(y, x) for x in range(0, self.x_size) ] for y in range(0,self.y_size) ]
        for y in range(0,self.y_size):
            self.maze[y][0] = Point(y, y, 0, self.river[y][0], 1)
        #print(self.maze)
    
    def create_init_point(self, y, x):
        return Point(-1, y, x, self.river[y][x], math.inf)
    
    def is_end_point(self, p):
        return p.x == self.x_size - 1
    
    def get_best_point(self, p1, p2):
        if not p2: return p1
        return p1 if ( p1.depth < p2.depth ) or ( p1.depth == p2.depth and p1.steps < p2.steps ) else p2
    
    def is_valid_point(self, p, step):
        y = p.y + step[0]
        x = p.x + step[1]
        return ( y >= 0 and y < self.y_size and x > 0 and x < self.x_size ) 

    def next_step(self, p, step): 
        y = p.y + step[0]
        x = p.x + step[1]
        d = p.depth if p.depth > self.maze[y][x].depth else self.maze[y][x].depth
        return Point(p.id, y, x, d, p.steps + 1) 
        
    def find_next_points(self, p):
        valid_pts = ( self.next_step(p, s) for s in STEPS if self.is_valid_point(p, s) )
        return filter(lambda np: np.steps < self.maze[np.y][np.x].steps, valid_pts)

    def find_prev_point(self, p):
        prev = None
        for step in STEPS:
            y = p.y + step[0]
            x = p.x + step[1]
            if y >= 0 and y < self.y_size and x >= 0 and x < self.x_size:
                pp = self.maze[y][x]
                if pp.depth <= p.depth and pp.steps < p.steps:
                    if not prev or pp.steps < prev.steps: prev = pp
        if not prev: 
            raise ValueError("Incorrect point")
        return prev
    
    def build_path(self, p):
        cp = p
        print(cp)
        res = [ ]
        while cp.x != 0:
            res.append([cp.y,cp.x])
            cp = self.find_prev_point(cp)
            print(cp)
        res.append([cp.y,cp.x])
        return res[::-1]

    def find(self):
        
        if self.x_size == 1: #check if 1 column maze
            cur_max_d = self.maze[0][0].depth
            path = [ [0,0] ]
            for y in range(1,self.y_size):
                if self.maze[y][0].depth < cur_max_d:
                    cur_max_d = self.maze[y][0].depth
                    path = [ [y,0] ]
            return path
            
        queue = PathQueue()
        
        for y in range(0,self.y_size):
            queue.push(self.maze[y][0])
            #self.cur_max_d = min(self.cur_max_d, self.maze[y][0].depth)
            
        best_point = None
        while queue.is_not_empty():
            cp = queue.pop()
            if not best_point or cp.depth < best_point.depth:
                next_pts = self.find_next_points(cp)
                for np in next_pts:
                    self.maze[np.y][np.x] = np
                    if self.is_end_point(np):
                          best_point = self.get_best_point(np, best_point)
                    else:
                        queue.push(np)
                    
        print('done') 
        #print(self.maze) 
        print('--') 
        print(best_point) 
        print('--')
        return self.build_path(best_point);
    
    
def shallowest_path(river):
    if not river: return []
    finder = PathFinder(river)
    return finder.find()


In [71]:
river = [
    [2, 3, 2],
    [1, 1, 4],
    [9, 5, 2],
    [1, 4, 4],
    [1, 5, 4],
    [2, 1, 4],
    [5, 1, 2],
    [5, 5, 5],
    [8, 1, 9]
]

shallowest_path(river)

done
--
Point(id=1, y=2, x=2, depth=2, steps=3)
--
Point(id=1, y=2, x=2, depth=2, steps=3)
Point(id=1, y=1, x=1, depth=1, steps=2)
Point(id=1, y=1, x=0, depth=1, steps=1)


[[1, 0], [1, 1], [2, 2]]

In [72]:
river = [
[8],
[8],
[8],
[8],
[8],
[8],
[5],
[8],
[8],
[8],
[8],
[8]]

shallowest_path(river)

[[6, 0]]

In [73]:
river = [
    [76, 30, 68, 95, 99, 48, 93, 49, 94, 55, 33, 33, 94, 28, 63], 
    [90, 94, 81, 86, 88, 27, 81, 1, 91, 53, 74, 18, 24, 13, 78], 
    [69, 61, 73, 75, 33, 6, 82, 15, 99, 12, 54, 56, 43, 83, 77], 
    [74, 34, 1, 11, 19, 55, 8, 12, 82, 24, 97, 44, 72, 50, 60], 
    [93, 78, 64, 38, 88, 14, 40, 62, 6, 61, 86, 89, 37, 51, 61], 
    [57, 81, 58, 35, 54, 76, 32, 63, 19, 25, 48, 81, 56, 52, 84], 
    [41, 31, 55, 36, 47, 60, 49, 15, 17, 25, 92, 50, 2, 75, 96], 
    [10, 44, 96, 35, 20, 50, 26, 11, 87, 40, 40, 43, 89, 44, 2], 
    [87, 19, 30, 73, 71, 24, 42, 25, 44, 31, 3, 85, 22, 86, 32], 
    [49, 82, 96, 80, 83, 97, 38, 60, 68, 11, 8, 23, 63, 58, 1], 
    [85, 79, 74, 13, 53, 57, 19, 81, 55, 97, 61, 10, 94, 1, 38], 
    [89, 29, 48, 35, 72, 59, 29, 51, 89, 55, 66, 71, 42, 28, 92], 
    [10, 6, 47, 45, 36, 35, 33, 17, 73, 42, 24, 99, 24, 13, 78], 
    [73, 77, 44, 60, 74, 90, 38, 37, 98, 27, 18, 8, 55, 69, 87], 
    [97, 7, 14, 41, 74, 38, 64, 22, 54, 22, 21, 42, 42, 45, 57]
]
shallowest_path(river)

done
--
Point(id=7, y=10, x=14, depth=42, steps=16)
--
Point(id=7, y=10, x=14, depth=42, steps=16)
Point(id=7, y=11, x=13, depth=42, steps=15)
Point(id=7, y=11, x=12, depth=42, steps=14)
Point(id=7, y=10, x=11, depth=40, steps=13)
Point(id=7, y=9, x=11, depth=40, steps=12)
Point(id=7, y=8, x=10, depth=40, steps=11)
Point(id=7, y=7, x=9, depth=40, steps=10)
Point(id=7, y=6, x=8, depth=35, steps=9)
Point(id=7, y=7, x=7, depth=35, steps=8)
Point(id=7, y=7, x=6, depth=35, steps=7)
Point(id=7, y=8, x=5, depth=35, steps=6)
Point(id=7, y=7, x=4, depth=35, steps=5)
Point(id=7, y=7, x=3, depth=35, steps=4)
Point(id=7, y=8, x=2, depth=30, steps=3)
Point(id=7, y=8, x=1, depth=19, steps=2)
Point(id=7, y=7, x=0, depth=10, steps=1)


[[7, 0],
 [8, 1],
 [8, 2],
 [7, 3],
 [7, 4],
 [8, 5],
 [7, 6],
 [7, 7],
 [6, 8],
 [7, 9],
 [8, 10],
 [9, 11],
 [10, 11],
 [11, 12],
 [11, 13],
 [10, 14]]